In [1]:
from sklearn.datasets import load_diabetes
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import random

In [2]:
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [7]:

print(X.shape)
print(y.shape)

(442, 10)
(442,)


In [4]:
class MBGD:
    def __init__(self, learning_rate=0.01, epochs=100, batch_size=32):
        self.lr = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.coef_ = None
        self.intercept_ = None
        
    def fit(self, X, y):
        # Initialize parameters
        n_samples, n_features = X.shape
        self.coef_ = np.zeros(n_features)
        self.intercept_ = 0
        
        for epoch in range(self.epochs):
            # Shuffle data each epoch
            indices = np.arange(n_samples)
            np.random.shuffle(indices)
            
            # Process in mini-batches
            for i in range(0, n_samples, self.batch_size):
                batch_indices = indices[i:i+self.batch_size]
                X_batch = X[batch_indices]
                y_batch = y[batch_indices]
                
                # Calculate predictions
                y_pred = np.dot(X_batch, self.coef_) + self.intercept_
                
                # Compute gradients
                error = y_batch - y_pred
                grad_coef = -(2/X_batch.shape[0]) * np.dot(X_batch.T, error)
                grad_intercept = -2 * np.mean(error)
                
                # Update parameters
                self.coef_ -= self.lr * grad_coef
                self.intercept_ -= self.lr * grad_intercept
    
    def predict(self, X):
        return np.dot(X, self.coef_) + self.intercept_


In [5]:
mbgd = MBGD(learning_rate=0.1, epochs=50, batch_size=32)
mbgd.fit(X_train, y_train)

In [6]:
# Predict and evaluate
y_pred = mbgd.predict(X_test)
print("R2 Score:", r2_score(y_test, y_pred))
print("Coefficients:", mbgd.coef_)
print("Intercept:", mbgd.intercept_)

R2 Score: 0.33634517410389786
Coefficients: [  57.99757974   -9.02833348  186.77368142  146.4482943    43.95910843
   22.68579766 -103.97457716  101.34661023  179.52942494   98.47528307]
Intercept: 160.87350569161774


In [8]:
# Compare with scikit-learn's SGD
from sklearn.linear_model import SGDRegressor
sgd = SGDRegressor(learning_rate='constant', eta0=0.1, max_iter=50, 
                   tol=None, random_state=2, average=32)
sgd.fit(X_train, y_train)


SGDRegressor(average=32, eta0=0.1, learning_rate='constant', max_iter=50,
             random_state=2, tol=None)

In [9]:
print("\nScikit-learn SGD Results:")
print("R2 Score:", r2_score(y_test, sgd.predict(X_test)))
print("Coefficients:", sgd.coef_)
print("Intercept:", sgd.intercept_)


Scikit-learn SGD Results:
R2 Score: 0.4522478046758025
Coefficients: [  25.46390456 -131.46718118  419.16007323  281.28952499  -18.90233413
  -78.74856755 -174.65323849  102.66031976  387.74837848   99.61802216]
Intercept: [151.91762266]
